# 1st test on Whisper
look faster Whisper

In [2]:
import whisper
import os

In [3]:
model = whisper.load_model("base")

c:\Users\loicm\miniconda3\envs\DeepLearningProject\lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_l

In [32]:
audio_path = "../../dataset/non_hate_audios_clean/non_hate_video_3.wav"

# check if the audio path is valid
if not os.path.exists(audio_path):
    print("Audio file not found")


In [33]:
result = whisper.transcribe(model, audio_path)
print(result)

{'text': " In Lehigh County authorities released the disturbing details of a crime the district attorney calls Repugnant a 22 year old man as accused of targeting and sexually assaulting children at a daycare center in Whitehall township Highwood Institute's important map of Trillow is there and Matt that suspect was just arraigned this morning That's right. He's charged with TV its sexual intercourse of a child authorities tell us there's assaults happen at this daycare behind me over several years Did Zimmerman work here? No. Well, when he answers the door at a lot like home daycare near Almond Town says 22 year old Zachary Zimmerman did not work there before slamming the door. We approve the charges But Lehigh County District Attorney Jim Martin says Zimmerman sexually assaulted three girls there. I witnessed news found the daycare was located at the home where Zimmerman lived. Anytime a child is a victim of sexual assault it's it's repugnant It's unfortunate obviously into crime Zi

In [34]:
# Get full transcription text
full_text = result["text"]
print("Full transcription:")
print(full_text)

Full transcription:
 In Lehigh County authorities released the disturbing details of a crime the district attorney calls Repugnant a 22 year old man as accused of targeting and sexually assaulting children at a daycare center in Whitehall township Highwood Institute's important map of Trillow is there and Matt that suspect was just arraigned this morning That's right. He's charged with TV its sexual intercourse of a child authorities tell us there's assaults happen at this daycare behind me over several years Did Zimmerman work here? No. Well, when he answers the door at a lot like home daycare near Almond Town says 22 year old Zachary Zimmerman did not work there before slamming the door. We approve the charges But Lehigh County District Attorney Jim Martin says Zimmerman sexually assaulted three girls there. I witnessed news found the daycare was located at the home where Zimmerman lived. Anytime a child is a victim of sexual assault it's it's repugnant It's unfortunate obviously int

In [35]:
for seg in result["segments"]:
    print(f"[{seg['start']:.2f} - {seg['end']:.2f}] {seg['text']}")


[0.00 - 5.04]  In Lehigh County authorities released the disturbing details of a crime the district attorney calls
[5.36 - 12.40]  Repugnant a 22 year old man as accused of targeting and sexually assaulting children at a daycare center in Whitehall township
[12.80 - 17.76]  Highwood Institute's important map of Trillow is there and Matt that suspect was just arraigned this morning
[19.84 - 28.00]  That's right. He's charged with TV its sexual intercourse of a child authorities tell us there's assaults happen at this daycare behind me over several years
[28.00 - 40.88]  Did Zimmerman work here? No. Well, when he answers the door at a lot like home daycare near Almond Town says 22 year old Zachary Zimmerman did not work there before slamming the door. We approve the charges
[40.96 - 50.56]  But Lehigh County District Attorney Jim Martin says Zimmerman sexually assaulted three girls there. I witnessed news found the daycare was located at the home where Zimmerman lived.
[50.64 - 54.64]  A

ajouter ca au CSV -> 2 col

In [ ]:
# Post processing

# ???


In [36]:
from transformers import pipeline

# Hate Bert 
# Load the hate speech detection pipeline
hate_speech_detector = pipeline("text-classification", model="Hate-speech-CNERG/dehatebert-mono-english")


Device set to use cuda:0


In [37]:
# Run hate speech analysis on the full text
analysis = hate_speech_detector(full_text)
print("Full Text Analysis:", analysis)

Full Text Analysis: [{'label': 'NON_HATE', 'score': 0.8897497057914734}]


In [15]:
# Decide if it's hate speech
if analysis[0]["label"] == "HATE":
    print("\n⚠️ Full text was classified as containing HATE speech.")
    print("→ Now trying to locate hateful segments...")
    
    # Optional: check segments only *after* full text is flagged
    hate_segments = []
    for seg in result["segments"]:
        text = seg["text"]
        seg_analysis = hate_speech_detector(text)
        if seg_analysis[0]["label"] == "HATE":
            hate_segments.append((seg, seg_analysis))
            print(f"[{seg['start']:.2f} - {seg['end']:.2f}] {text}")
            print(f"→ Segment flagged as HATE speech: {seg_analysis}")
else:
    print("\n✅ Full text is NOT classified as hate speech.")


⚠️ Full text was classified as containing HATE speech.
→ Now trying to locate hateful segments...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


[17.20 - 20.72]  if that's how you solve it in a big black spute you know
→ Segment flagged as HATE speech: [{'label': 'HATE', 'score': 0.8409350514411926}]
[44.00 - 48.00]  Oh, I'm gonna slam all the poor nigger, hating me.
→ Segment flagged as HATE speech: [{'label': 'HATE', 'score': 0.9118627309799194}]
[84.00 - 86.00]  I'll follow the silver nigger, bro.
→ Segment flagged as HATE speech: [{'label': 'HATE', 'score': 0.9131659269332886}]
[100.00 - 102.00]  The niggers have.
→ Segment flagged as HATE speech: [{'label': 'HATE', 'score': 0.9139613509178162}]
[112.00 - 116.00]  Let's get rid of nigger, hating me.
→ Segment flagged as HATE speech: [{'label': 'HATE', 'score': 0.9114501476287842}]


tester avec les segment. et comparer clean pas clean

les segments 